In [1]:
!pip freeze | grep pyarrow

pyarrow==8.0.0


In [2]:
import pickle
import pandas as pd

In [3]:
year = 2021
month = 2

input_file = f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/fhv_tripdata_{year:04d}-{month:02d}.parquet'

In [4]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df, categorical

In [9]:
def add_id(df):
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    return df

In [6]:
def write_output(input_df, y_pred, output_file):
    df_result = pd.DataFrame()

    df_result['ride_id'] = input_df['ride_id']
    df_result['predicted_duration'] = y_pred

    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [10]:
def apply_model(input_file, model_path, output_file):
    df, categorical = read_data(input_file)

    with open(model_path, 'rb') as f_in:
        dv, lr = pickle.load(f_in)

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)

    print(f"Mean predicted duration: {y_pred.mean()}")

    df = add_id(df)
    write_output(input_df=df, y_pred=y_pred, output_file=output_file)

In [11]:
apply_model(input_file, './model.bin', output_file)

/home/kjoconnell/anaconda3/envs/exp_tracking/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/kjoconnell/anaconda3/envs/exp_tracking/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Mean predicted duration: 16.191691679979066


In [7]:
y_pred.mean()

16.191691679979066

In [13]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [17]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [18]:
!ls -lh output/

total 19M
-rw-rw-r-- 1 ubuntu ubuntu 19M Jun 30 08:43 fhv_tripdata_2021-02.parquet
